<center><h2>Example 1: Calculate the potential energy surface </h2> </center> <br>
---
    
    
This notebook builds the potential energy surface (PES) of pairs of molecules along characteristic coordinates. N-dimensional scans can be performed and plotted to show the PES.<br>
<br>

 <i>Author</i>: <a href="https://github.com/oosode">Olaseni Sode</a> <br>
 <i>Created on</i>: June 1, 2018 <br>
 <i>Tags</i>: PES, electronic structure<br>

# Contents

---
   - [0. Load imports](#0.-Load-imports)
   - [1. Load dfk and functions](#1.-Load-dfk-and-functions)
   - [2. Choose molecules](#2.-Choose-molecules)
   - [3. Pick theory](#3.-Pick-theory)
   - [4. Optimize monomers](#4.-Optimize-monomers)
   - [5. Configure dimer](#5.-Configure-dimer)
   - [6. Optimize dimer](#6.-Optimize-dimer)
   - [7. Setup PES scan](#7.-Setup-scan)
   - [8. PES scan](#8.-PES-scan)
   - [9. Plot PES](#9.-Plot-PES)

## 0. Load imports

This cell imports the necessary modules. To execute a cell, click on it, then press <kbd>shift</kbd> + <kbd>enter</kbd>. (If you're new to the notebook environment, you may want to check out [this helpful cheat sheet](https://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Notebook%20Basics.ipynb)).

In [ ]:
import os
import sys
import gc
import parsl
import py3Dmol
import types

import ipywidgets as widgets
from subprocess import Popen, PIPE
from parsl import *
from math import *
from copy import *
#from molecule import *

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import cirpy

%matplotlib inline

print(parsl.__version__)

## 1. Load dfk and functions

The following cells are used to load the data flow kernel need for parsl and the other necessary python functions needed to perform the potential energy surface scan. 

#### Load data flow kernel

In [ ]:
from parsl.configs.local import localThreads

config = {
    "sites": [
        {
            "site": "midway_ipp_multicore",
            "auth": {
                "channel": "local",
                "hostname": "swift.rcc.uchicago.edu",
                "scriptDir": "/scratch/midway2/{0}/parsl_scripts".format('oosode')
            },
            "execution": {
                "executor": "ipp", # remote executor
                "provider": "slurm", 
                "block": {
                    "nodes": 1, # nodes per block
                    "taskBlocks": 1, # tasks per block
                    #"taskBlocks":20, 
                    "walltime": "00:05:00",
                    "initBlocks": 1,
                    "minBlocks" : 1,
                    "maxBlocks": 10,
                    "options": {'partition' : 'westmere',
                                'overrides' : 'source activate python36' }
                }
            }
        }, 
        {
            "site": "midway_ipp_singlenode",
            "auth": {
                "channel": "local",
                "hostname": "swift.rcc.uchicago.edu",
                "scriptDir": "/scratch/midway2/{0}/parsl_scripts".format('oosode')
            },
            "execution": {
                "executor": "ipp", # remote executor
                "provider": "slurm", 
                "block": {
                    "nodes": 1, # nodes per block
                    #"taskBlocks": 1, # tasks per block
                    "taskBlocks":20, 
                    "walltime": "00:15:00",
                    "initBlocks": 1,
                    "minBlocks" : 1,
                    "maxBlocks": 1,
                    "options": {'partition' : 'westmere',
                                'overrides' : 'source activate python36' }
                }
            }
        }
        
    ],
    "globals": {
        "lazyErrors": True,
    },
    #"controller" : {'publicIp' : ''}
}

#dfk = DataFlowKernel(config=config)
dfk = DataFlowKernel(config=localThreads)

In [ ]:
import element

class molecule:
    def __init__(self,name,nmols):
        print("Intialized new molecule class: %s"%(name))
        self.name = name
        self.nmols = nmols
        self.coord = []
        self.mass = []
        self.atom = []
        self.frag = []

        return

    def resolve(self):
        res=cirpy.resolve(self.name, 'xyz')
        print("Resolved chemical identification of %s"%(self.name))
        r=res.split("\n")
        self.natoms=int(r[0])
        self.frag=[0]*self.natoms

        for i,line in enumerate(r):
            if i<=1:
                continue
            elif i<self.natoms+2:
                tmp=[float(line.split()[1]),
                     float(line.split()[2]),
                     float(line.split()[3])]
                self.coord.append(tmp)
                self.atom.append(line.split()[0])
                self.mass.append(element.ELEMENTS[self.atom[-1]].mass)
            else:
                continue
        return
    
    def init_dimer(self,m1,m2):
        
        
        self.name=m1.name+"-"+m2.name
        self.nmols=2
        self.coord=deepcopy(m1.coord)+deepcopy(m2.coord)
        self.mass=deepcopy(m1.mass)+deepcopy(m2.mass)
        self.natoms=m1.natoms+m2.natoms
        self.atom=deepcopy(m1.atom)+deepcopy(m2.atom)
        self.frag=[0]*m1.natoms+[1]*m2.natoms

        return
    
    def xyz(self):
        
        string="%d\n\n"%(self.natoms)
        for c,coord in enumerate(self.coord):
            string+="%-2s "%(self.atom[c])
            string+="%15.10f %15.10f %15.10f\n"%(coord[0],
                                                 coord[1],
                                                 coord[2])
            
        return string
        
    def print_xyz(self):
        
        string="%d\n\n"%(self.natoms)
        for c,coord in enumerate(self.coord):
            string+="%-2s "%(self.atom[c])
            string+="%15.10f %15.10f %15.10f\n"%(coord[0],
                                                 coord[1],
                                                 coord[2])
        print(string)
        return
    
    def calculate_com(self):
    
        if self.nmols==1:
            x=0.0
            y=0.0
            z=0.0

            totalmass=0.0

            for a,atom in enumerate(self.coord):

                x+=atom[0]*self.mass[a]
                y+=atom[1]*self.mass[a]
                z+=atom[2]*self.mass[a]
                totalmass+=self.mass[a]

            x=x/totalmass
            y=y/totalmass
            z=z/totalmass

            self.com=[x,y,z]

        elif self.nmols==2:

            x1=x2=0.0
            y1=y2=0.0
            z1=z2=0.0

            totalmass1=totalmass2=0.0

            for c,coord in enumerate(self.coord):

                if self.frag[c]==0:
                    x1+=coord[0]*self.mass[c]
                    y1+=coord[1]*self.mass[c]
                    z1+=coord[2]*self.mass[c]
                    totalmass1+=self.mass[c]
                elif self.frag[c]==1:
                    x2+=coord[0]*self.mass[c]
                    y2+=coord[1]*self.mass[c]
                    z2+=coord[2]*self.mass[c]
                    totalmass2+=self.mass[c]

            x1=x1/totalmass1
            y1=y1/totalmass1
            z1=z1/totalmass1
                   
            x2=x2/totalmass2
            y2=y2/totalmass2
            z2=z2/totalmass2

            self.com=[x1,y1,z1,x2,y2,z2]
                   
        return
   
    def calculate_vector(self):
        """Function to calculate normalized vector."""

        self.calculate_com()
        
        for l in range(3):
            x=self.com[3]-self.com[0]
            y=self.com[4]-self.com[1]
            z=self.com[5]-self.com[2]
            
        norm=sqrt(x*x + y*y + z*z)
        self.vector=[x/norm,y/norm,z/norm]
        self.radius=norm
        
        return

    
    def center(self):

        self.calculate_com()
        f=self.frag
                   
        for c,coord in enumerate(self.coord):
            coord[0]-=self.com[0+f[c]*3]
            coord[1]-=self.com[1+f[c]*3]
            coord[2]-=self.com[2+f[c]*3]         

        return

#     def rotate_theta(self,theta,mID):

#         if mID!=0 and self.nmols==1:
#             print("This is a single molecule. The molecule ID should be zero.")  
            
#         for c,coord in enumerate(self.coord):
            
#             if self.frag[c]==mID:
#                 x=coord[0]
#                 y=coord[1]
#                 z=coord[2]

#                 r=sqrt(x*x + y*y + z*z)
#                 if r<1.0e-10:
#                     continue
                    
#                 #t=acos(z/r)
#                 t=atan2(r,z)
#                 print(t,t-radians(theta))
#                 p=atan2(y,x)
#                 print()
#                 t+=radians(theta)

#                 coord[0]=r*sin(t)*cos(p)
#                 coord[1]=r*sin(t)*sin(p)
#                 coord[2]=r*cos(t)

#         return
    
#     def rotate_phi(self,phi,mID):
        
#         if mID!=0 and self.nmols==1:
#             print("This is a single molecule. The molecule ID should be zero.")  
            
#         for c,coord in enumerate(self.coord):
            
#             if self.frag[c]==mID:
#                 x=coord[0]
#                 y=coord[1]
#                 z=coord[2]

#                 r=sqrt(x*x + y*y + z*z)
#                 if r<1.0e-10:
#                     continue
                    
#                 #t=acos(z/r)
#                 t=atan2(r,z)
#                 p=atan2(y,x)
#                 p+=radians(phi)

#                 coord[0]=r*sin(t)*cos(p)
#                 coord[1]=r*sin(t)*sin(p)
#                 coord[2]=r*cos(t)

#         return    
    
    def rotate(self,angle,axis,mID):
        """https://en.wikipedia.org/wiki/Rodrigues%27_rotation_formula"""
        
        if mID!=0 and self.nmols==1:
            print("This is a single molecule. The molecule ID should be zero.")  
        
        if axis=="x":
            k=(1,0,0) #unit vector
        elif axis=="y":
            k=(0,1,0)
        elif axis=="z":
            k=(0,0,1)
        else:
            if isinstance(var, types.StringTypes) or len(axis)!=3:
                 raise IOError("Unknown axis in rotate function.")
            else:
                norm=np.linalg.norm(axis)
                k=(axis[0]/norm,axis[1]/norm,axis[2]/norm)
                
        cosA=cos(radians(angle)) # cosine of angle
        sinA=sin(radians(angle)) # sine of angle
    
        for c,coord in enumerate(self.coord):
            
            if self.frag[c]==mID:
                
                term0=[l*cosA for l in coord]
                
                kxv=np.cross(k,coord).tolist() # cross of k and v
                term1=[l*sinA for l in kxv]
                
                kv=np.dot(k,coord).tolist()
                term2=[l*(1-cosA)*kv for l in k]

                for l in range(3):
                    coord[l]=term0[l]+term1[l]+term2[l]

        return        

    def position(self,R,theta,phi):
        
        if self.nmols==1:
            print("This is a single molecule. The `position_molecule` function is for dimers.")        

        x=R*sin(radians(theta))*cos(radians(phi))
        y=R*sin(radians(theta))*sin(radians(phi))
        z=R*cos(radians(theta))

        for c,coord in enumerate(self.coord):
            
            if self.frag[c]==1:
                coord[0]+=x
                coord[1]+=y
                coord[2]+=z

        return    
    
    def setup_dimer(self):
        """This function that each monomer is initially at the origin, before
        transformation."""
        
        self.calculate_com()
        #self.R=0.0 # initial dimer distance
        #self.tv=[0.0]*3 # initial dimer vector
        self.center()
        return
        
    def transform_dimer(self,value,transform,unit,mID):
        """This function prepares the dimer according to the type and magnitude 
        of transformation. Since translations shift the dimer off the origin, this
        operation is always perfomed last in the `set dimer` function."""
        
        if transform=="translation": # make translational vector
            self.radius=value
            if unit==False:
                pass
            else:
                self.vector=unit
            
        elif transform=="angle": # rotate molecules
            
            norm=np.linalg.norm(unit)
            k=(unit[0]/norm,unit[1]/norm,unit[2]/norm)
                
            cosA=cos(radians(value)) # cosine of angle
            sinA=sin(radians(value)) # sine of angle
    
            for c,coord in enumerate(self.coord):
            
                if self.frag[c]==mID:
                
                    term0=[l*cosA for l in coord]
                
                    kxv=np.cross(k,coord).tolist() # cross of k and v
                    term1=[l*sinA for l in kxv]
                
                    kv=np.dot(k,coord).tolist()
                    term2=[l*(1-cosA)*kv for l in k]

                    for l in range(3):
                        coord[l]=term0[l]+term1[l]+term2[l]
            
        else:
            raise IOError("Unknown transformation.")
            
        return
            
    def set_dimer(self):
        """"""
        
        for c,coord in enumerate(self.coord):
            if self.frag[c]==1:
                coord[0]+=self.vector[0]*self.radius
                coord[1]+=self.vector[1]*self.radius
                coord[2]+=self.vector[2]*self.radius
                
        return



#### Function to write electronic structure input files

In [ ]:
def write_input(ID,mol,theory,basis,program,optimization,f12,cp):
    
    natoms=int(mol.natoms)
    
    if program=="MOLPRO":
        
        fout=open("%s.%06d.com"%("molpro",ID),"w")
        fout.write("***, %s %s \n"%(mol.name,optimization))
        fout.write("memory,100,M\n")
        fout.write("geometry={\n")
        
        for i in range(natoms):

            fout.write("%s%d "%(mol.atom[i],i))
            fout.write("%15.10f, %15.10f, %15.10f"%(mol.coord[i][0],\
                                                    mol.coord[i][1],\
                                                    mol.coord[i][2]))
            if i==natoms-1:
                fout.write("}\n")
            else:
                fout.write("\n")

        fout.write("basis=%s"%(basis))
        fout.write("\n")

        if cp:
            #This functionality doesn't exist yet.
            pass
        #    fout.write("text,cp calculation for %s %dmer_%i\n"%(name,m,n))
        #    fout.write("dummy,")
        #    for i in range(atoms):
        #        if molid[i] in dum:
        #            tmp=index*atoms
        #            fout.write("%s%d,"%(names[tmp+i],i))
        #            fout.write("\n")
        #        else:
        #            fout.write("text,calculation for %s %dmer\n"%(name,m))

        if theory!="hf":
            fout.write("hf\n")
            fout.write("%s"%(theory))
            if f12:
                fout.write("-f12\n")
            else:
                fout.write("\n")
        else:
            fout.write("hf\n")
            
        if optimization=="Minimum":
            fout.write("optg\n")
            fout.write("maxit=50\n")
        if optimization=="Saddle":
            fout.write("optg")

        fout.write("e_mer=energy\n")
        if (f12):
            fout.write("e_mer_F12a=energy(1)\n")
            fout.write("e_mer_F12b=energy(2)\n")
        fout.write("\n")
        fout.close()
        return
        
    if program=="NWChem":
        pass

#### Function to run electronic structure program

In [ ]:
@App('bash', dfk)
def molpro(ID, outputs=[], stderr=None, stdout=None):
    #return "uname -a; /home/oosode/software/molpro-2015/bin/molpro.exe  molpro.{0:06d}.com".format(ID)
    return "uname -a; /Users/oosode/Software/molpro-2015/bin/molpro.exe  molpro.{0:06d}.com".format(ID)

#### Function to read electronic structure ouptut

In [ ]:
def read_output(ID,program,mol):
    
    energy={}
    
    if program=="MOLPRO": 
        
        c=False
        fout="molpro.%06d.out"%(ID)
        if not os.path.isfile(fout):
            raiseIOError("No file for molpro.%06d.com"%(ID))

        else:
            fl=open(fout).readlines()

            for l in range(len(fl)-1,len(fl)-5,-1):
                if fl[l].find("Variable memory released")>=0:
                    c=True
                    break

            if (not c):
                energy[0]="inf"
                print("No energies for file molpro.%06d.com"%(ID))
                return

            else:
                for l,line in enumerate(fl):
                    if line.find("SETTING E_MER ")>=0: # energy
                        #print("hello")
                        energy['simple']=float(line.strip().split()[-2].replace("D","E"))
                    
                        mol.energy=energy
                        #print(mol['energy'])
                        
                    #if line.find("SETTING E_MER_F12A ")>=0: # F12a energy
                    #    energy['f12a']=float(line.strip().split()[-2].replace("D","E"))
                    #    mol['energy']=energy
                    #elif line.find("SETTING E_MER_F12B ")>=0: # F12b energy
                    #    energy['f12b']=float(line.strip().split()[-2].replace("D","E"))
                    #    mol['energy']=energy
                    
                    elif line.find("END OF GEOMETRY OPTIMIZATION.")>=0:
                        natoms=int(fl[l+4].split()[0])
                        if natoms != mol.natoms:
                            print("There is a mismatch in the number of atoms")
                        for n in range(natoms):
                            for i in range(3):
                                mol.coord[n][i]=float(fl[l+6+n].split()[i+1])
                    
            return


#### Function to setup the PES scan.

In [ ]:
def PES_scan(dim,variables,nCUP):
    
    # do pes scan
    sList = [] # list dimers to scan
    
    if nCUP==1:
        for m in range(len(variables)):
            
            var0=deepcopy(variables[m])
            
            for v0 in iter(deepcopy(var0)):
                
                tmp=deepcopy(dim) # copy dimer
                tmp.setup_dimer() 
                tmp.transform_dimer(v0,var0.name,var0.unit,var0.molID)
                tmp.set_dimer()
                sList.append(tmp)
                
    elif nCUP==2:
        for m in range(len(variables)):
            for n in range(m,len(variables)):
                
                var0=deepcopy(variables[m])
                var1=deepcopy(variables[n])
                
                for v0 in iter(var0):
                    for v1 in iter(var1):
                        
                        tmp=deepcopy(dim) # copy dimer
                        tmp.setup_dimer() 
                        tmp.transform_dimer(v0,var0.name,var0.unit,var0.molID)
                        tmp.transform_dimer(v1,var1.name,var0.unit,var1.molID)
                        tmp.set_dimer()
                        sList.append(tmp)
    
    else:
        print("The coupling between variables (ncup) must be set to either 0 or 1.")
                
    print(len(sList))      
    return sList

#### Class that defines the different variables.

In [ ]:
class Var:
    def __init__(self,name,minimum,maximum,inc,unit,mID=1):
        self.name=name
        self.min=minimum
        self.max=maximum
        self.inc=inc
        self.unit=unit
        self.molID=mID
        self.current=self.min
        return
        
    def determine_range(self):
        x=self.min
        tmp=[]
        while x<=self.max:
            tmp.append(x)
            x+=self.inc
        self.ranges=tuple(tmp)
        return
    
    def __iter__(self):
        return self

    def __next__(self):
        if self.current > self.max:
            raise StopIteration
        else:
            self.current += self.inc
            return self.current - self.inc
        

## 2. Choose molecules

The cells below allows you to pick the two types of molecules for the PES scan. It uses the cirpy module to resolve the name of the molecules and then loads the xyz coordinates and atom names into the `molecule` class. This class then configures the remaining values including atomic mass, number of atoms, etc. 

**NOTE:** You can change the molecule types from the default "carbon dioxide" to any other molecule in the chemical information resolver database. Just change the name below.

In [ ]:
mol0 = molecule("CO2",1) # name and number of molecules
mol0.resolve()

mol1 = molecule("H2O",1)
mol1.resolve()

mList = (mol0,mol1)

In [ ]:
view0 = py3Dmol.view(width=800,height=400,linked=False,viewergrid=(1,2))
#view0.removeAllModels(viewer=(0,0))
view0.addModel(mol0.xyz(),'xyz',viewer=(0,0))
view0.addModel(mol1.xyz(),'xyz',viewer=(0,1))
view0.setStyle({'sphere':{}})
view0.setBackgroundColor('0xeeeeee')
view0.zoomTo(viewer=(0,0))
#view0.render()
view0.show()

## 3. Pick theory

The cells below allows you to choose the level of electronic structure theory, the basis set, the use of explicit correlation and the electronic structure program. (Currently, only the MOLPRO software package is implemented.) 

Some configurations of theory, basis set and programs are not possible. Once you have selected your program variables, make sure to check the configurations availability with the `wrapper.check()` function.


In [ ]:
theory="HF"
basis="aug-cc-pVDZ"
f12=False
program="MOLPRO"

## 4. Optimize monomers

You can optimize the two monomers in the cells below. In the first cell, the input files are written. The second cell submits the input files and waits for them to finish. The last cell parses the output file and recovers the data. When running jobs on a compute cluster, the status of the job can be checked using the `qstat` command. The final energy and cartesian coordinates of the geometry are printed. 

**Write input files**:

In [ ]:
for ID in range(2):
    write_input(ID,mList[ID],theory,basis,program,"Minimum",f12,False)

**Run calculations**:

In [ ]:
futures=[]
for ID in range(2):
    #run_input(ID,program)
    out="molpro.%06d.out"%(ID)
    xml="molpro.%06d.xml"%(ID)
    futures.append(molpro(ID, outputs=[out,xml]))


print([x.result() for x in futures]) 

**Check status**:

In [ ]:
#!qstat -u $USER
!ls -lt | head -10
#!tail molpro.000000.out

**Read output**:

In [ ]:
for ID in range(2):
    read_output(ID,program,mList[ID])

## 5. Configure dimer position

Once the isolated monomers are obtained (and optimized), the next step is to orient them with respect to one another. This is done using a spherical coordinate space, shown below. The __orientation of molecule 1__ and the __orientation of molecule 2__ are determined first, followed by the __position__ of molecule 2 relative to molecule 1. 

Molecule 1 is always placed at the origin (0,0,0) oriented in space with $\theta_1$ and $\phi_1$. The range of $\theta_1$ is [0°, 180°] and the range of $\phi_1$ is [0, 360°). Molecule 2 is placed using spherical coordinates at some distance R from the origin, with $\theta_d$ and $\phi_d$ ranging from [0°, 180°] and [0, 360°), respectively. The orientation of Molecule 2 is denoted by $\theta_2$ and $\phi_2$ values. All angles are in degree units and distances are in angstroms.

<img src='https://upload.wikimedia.org/wikipedia/commons/4/4f/3D_Spherical.svg' style="width: 300px;">

The molecules are oriented such that the x-axis runs horizontally across the screen, the y-axis runs vertically, and the z-axis emerges from the plane of the computer screen.

#### Orientation and position molecules

In [ ]:
# orientation of molecule 1
x1_rotation = 0
y1_rotation = 90
z1_rotation = 0

# orientation of molecule 2
x2_rotation = 0
y2_rotation = 135
z2_rotation = 0

# make monomer list
xList = (x1_rotation,x2_rotation)
yList = (y1_rotation,y2_rotation)
zList = (z1_rotation,z2_rotation)

# position molecule 2
R = 4.0
theta_d = 90
phi_d = -90

#### Orient dimer

In [ ]:
tmp_mList=[]
for i,m in enumerate(mList):
    
    tmp_m = deepcopy(m)
    tmp_m.calculate_com()
    tmp_m.center()
    tmp_m.rotate(xList[i],"x",0)
    tmp_m.rotate(yList[i],"y",0)
    tmp_m.rotate(zList[i],"z",0)
    tmp_mList.append(tmp_m)

dimer=molecule("whatever",1)
dimer.init_dimer(tmp_mList[0],tmp_mList[1])
dimer.position(R,theta_d,phi_d)
dimer.calculate_vector()


#### Show dimer

In [ ]:
view1 = py3Dmol.view(width=800,height=300)
view1.addModel(dimer.xyz(),'xyz')
view1.setStyle({'stick':{}})
view1.setBackgroundColor('0xeeeeee')
view1.show()

If you are unhappy with the orientation of the dimer, you may return to the `Orientation and position of molecules` cell and adjust the angles and radial distances to your liking. Also, one should consider optimizing the dimer to either a local minimum or saddle point in the following cell.

## 6. Optimize dimer structure

The dimer structure can be optimized such that any potential energy scan originates from either a global or local minimum, or a saddle point. To optimize to a minimum, set the optimization flag to `Minimum`, and to optimize to a saddle point, set the optimization variable to `Saddle`. If you prefer not to optimize the dimer further, specify `None`.



In [ ]:
optimization="None"

In [ ]:
if optimization=="None":
    pass

elif optimization=="Minimum" or "Saddle":
    write_input(3,dim,theory,basis,program,optimization,f12,False)
    fut = molpro(3, outputs=["molpro.%06d.out"%(3), "molpro.%06d.xml"%(3)],
                     stdout="molpro.%06d.stdout"%(3),
                     stderr="molpro.%06d.stderr"%(3))
    read_output(3,program)
    
else:
    raise IOError("Unknown optimization.")

## 7. Setup scan

The potential energy surface will be scanned along some characteristic coordinate(s). There is no limit to the number coordinates to scan however, the maximum coupling between coordinates is 2. Some examples of coordinates include the 'R' distance, the $\phi_1$ angle or the $\theta_d$ angle. For each coordinate, the range of values and the increment must be specified. Increments for distances are in angstroms and are in degrees for angles. 

**Which variables?**

In [ ]:
variables=[]

Rname = 'translation'
Rmini = 2.0
Rmaxi = 8.0
Rincr = 0.1
Runit = False # scan along what vector from molecule 1. False is distance vector
RID = 1 # this value should always be one for translations

var1=Var(Rname,Rmini,Rmaxi,Rincr,Runit,1)
var1.determine_range()
#variables.append(var1)

Aname = 'angle'
Amini = -90.0
Amaxi =  90.0
Aincr =  10.0
Aunit = (1,0,0) # rotate about what axis 
AID = 0

var2=Var(Aname,Amini,Amaxi,Aincr,Aunit,AID)
var2.determine_range()
variables.append(var2)

**number of variable coupling?**

In [ ]:
nCUP=1

## 8. PES scan

Once the scan variables are determined, proceed through the next cells to perform electronic structure calculations on each of the dimer configurations.

#### Determine dimer configurations

This also prints the number of configurations to evaluate.

In [ ]:
dimer_scans=PES_scan(dimer,variables,nCUP)

#### Write scan input files

In [ ]:
for k,dscan in enumerate(dimer_scans): # loop over monomers
    write_input(k+3,dscan,theory,basis,program,"None",f12,False)

#### Run calculations

In [ ]:
futures=[]
for k in range(len(dimer_scans)):
    #run_input(k+3,program)
    out="molpro.%06d.out"%(k+3)
    xml="molpro.%06d.xml"%(k+3)
    futures.append(molpro(k+3, outputs=[out,xml]))

print([x.result() for x in futures]) 

In [ ]:
#!qstat -u $USER
! ls -lt | head -10


#### Read outputs and collect energies

In [ ]:
x=[]
nrg=[]
for k,dscan in enumerate(dimer_scans): # loop over dimers
    read_output(k+3,program,dscan)
    nrg.append(dscan.energy['simple'])


## 9. Plot PES



In [ ]:
%matplotlib inline
plt.figure()
plt.title("Dimer energies", size='xx-large')
plt.ylabel("total energy / hartrees", size='x-large')
plt.ylim([-263.71,-263.70])
plt.plot(variables[0].ranges,nrg)